In [2]:
import pandas as pd

## 레진 데이터 챌린지 2017

오늘의 주제는 [레진 데이터 챌린지 2017](http://tech.lezhin.com/events/data-challenge-pyconkr-2017)의 데이터를 바탕으로, 레진코믹스의 구매량과 주 사용 플랫폼과 작품 등의 상관관계를 분석하는 것입니다. 총 데이터는 650,965 건이며, 이 데이터를 판다스로 활용해 분석할 수 있어야 합니다.

[원래 경진대회](http://tech.lezhin.com/events/data-challenge-pyconkr-2017) 에서는 머신러닝을 활용해 구매 예측 여부를 판단하였지만, 이번 과제에서는 머신러닝을 사용하지 않고, 판다스만을 사용해 분석할 것입니다.

데이터는 [다음의 링크](https://storage.googleapis.com/lz-insight/pycon17/dataset/lezhin_dataset_v2_training.tsv.gz)에서 다운받으실 수 있습니다.

** 목표 **

주어진 데이터를 활용하여, 타 팀에서 요청한 10여개의 질문에 대한 답변을 줘야 합니다. 툴은 파이썬과 판다스를 사용하며, 타 팀에서 직관적으로 이해할 수 있도록 가능한 깔끔하고 정리된 결과가 나오면 좋습니다.

** 데이터셋 **
* 파일 포맷: TSV
* 파일 용량: 228M (압축해서 26M)
* 샘플 수: 650,965 건
* feature 수: 167 개
  * 1 : label. 해당 유저가 목록에 진입하고 1시간 이내에 구매했는지 여부
  * 2 : 사용 플랫폼 A
  * 3 : 사용 플랫폼 B
  * 4 : 사용 플랫폼 C
  * 5 : 사용 플랫폼 D
  * 6 : 목록 진입시점 방문 총 세션 수 (범위별로 부여된 순차 ID)
  * 7 : 작품을 나타내는 해쉬
  * 8-10 : 개인정보
  * 11-110 : 주요 작품 구매 여부
  * 111 : 작품 태그 정보
  * 112 : 구매할 때 필요한 코인
  * 113 : 완결 여부
  * 114-123 : 스케쥴 정보
  * 124-141 : 장르 정보
  * 142 : 해당 작품의 마지막 에피소드 발행 시점 (범위별로 부여된 순차 ID)
  * 143 : 단행본 여부
  * 144 : 작품 발행 시점 (범위별로 부여된 순차 ID)
  * 145 : 총 발행 에피소드 수 (범위별로 부여된 순차 ID)
  * 146-151 : 작품 태그 정보
  * 152-167 : 유저의 성향 정보 (과거에 구매를 했을 때만 기록)

## 데이터 로딩하기

In [3]:
data = pd.read_csv("lezhin_dataset_v2_training.tsv", sep="\t", header=None)

print(data.shape)
data.head()

(650965, 167)


,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,0,0,0,0,1,1,001C9D9B,8337B6FB,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.2580,0.0030,NaN,NaN
1,0,0,0,0,1,1,001C9D9B,90D8AB70,1,NaN,...,NaN,0.5000,NaN,NaN,NaN,NaN,0.5000,NaN,NaN,NaN
2,0,0,0,0,1,1,001C9D9B,ABC21E80,1,NaN,...,NaN,0.0187,0.0047,NaN,NaN,NaN,0.0287,0.0055,NaN,NaN
3,0,0,0,0,1,1,001C9D9B,C17967D1,0,69EF2C8F,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0186,NaN,NaN,NaN
4,0,0,0,0,1,1,002B4BDE,AF145784,0,NaN,...,NaN,0.0207,NaN,NaN,NaN,NaN,0.2805,0.0692,NaN,NaN


### 1. 사용 플랫폼 분석

다음의 데이터를 활용하여 insights를 얻고 싶습니다.

  * 1 : label. 해당 유저가 목록에 진입하고 1시간 이내에 구매했는지 여부 (이하 구매 여부)
  * 2 : 사용 플랫폼 A
  * 3 : 사용 플랫폼 B
  * 4 : 사용 플랫폼 C
  * 5 : 사용 플랫폼 D

In [2]:
platform = data[[0, 1, 2, 3, 4]].copy()
platform.columns = ["구매 여부", "사용 플랫폼 A", "사용 플랫폼 B", "사용 플랫폼 C", "사용 플랫폼 D"]

print(platform.shape)
platform.head()

(650965, 5)


,구매 여부,사용 플랫폼 A,사용 플랫폼 B,사용 플랫폼 C,사용 플랫폼 D
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1


** 1. 전체 데이터(650,965 건)에서 구매한 사용자 / 구매하지 않은 사용자의 인원 수를 찾아주세요. **

In [3]:
platform["구매 여부"].value_counts()

0    429172
1    221793
Name: 구매 여부, dtype: int64

** 2. 사용 플랫폼(A~D)별 구매한 사용자 / 구매하지 않은 사용자의 총 인원 수를 찾아주세요. . **

즉, 표의 세로축에는 구매 여부(0, 1)가, 가로축에서는 각 사용 플랫폼(A~D)별 구매한 사용자 / 구매하지 않은 사용자의 총 인원 수가 나왔으면 좋겠습니다.

In [4]:
pd.pivot_table(platform,
               index='구매 여부',
               values=["사용 플랫폼 A", "사용 플랫폼 B", "사용 플랫폼 C", "사용 플랫폼 D"],
               aggfunc=[np.sum])

sum                           
      사용 플랫폼 A 사용 플랫폼 B 사용 플랫폼 C 사용 플랫폼 D
구매 여부                                    
0        74685   149623    63049   141815
1        15108    42731    45023   118931

** 3. 2번 데이터에서 총 인원수는 물론, 평균 구매율도 함께 계산해주세요.**

힌트: 총 인원 수를 sum이라고 한다면, 평균 구매율은 mean이라고 할 수 있습니다.



In [5]:
pd.pivot_table(platform,
               index='구매 여부',
               values=["사용 플랫폼 A", "사용 플랫폼 B", "사용 플랫폼 C", "사용 플랫폼 D"],
               aggfunc=[np.sum, np.mean])

sum                                 mean                      \
      사용 플랫폼 A 사용 플랫폼 B 사용 플랫폼 C 사용 플랫폼 D  사용 플랫폼 A  사용 플랫폼 B  사용 플랫폼 C   
구매 여부                                                                     
0        74685   149623    63049   141815  0.174021  0.348632  0.146908   
1        15108    42731    45023   118931  0.068118  0.192662  0.202996   

                 
       사용 플랫폼 D  
구매 여부            
0      0.330439  
1      0.536225

** 4. '사용 플랫폼' 이라는 새로운 컬럼을 만들어주세요 **

데이터를 분석한 결과, 두 개 이상의 플랫폼에서 동시에 방문하는 경우는 찾을 수 없었습니다.

In [6]:
# 동시에 두 개 이상의 플랫폼에서 방문 + 구매하는 경우(> 1)는 없다는 것을 확인할 수 있다.
((platform["사용 플랫폼 A"] + platform["사용 플랫폼 B"] + platform["사용 플랫폼 C"] + platform["사용 플랫폼 D"]) > 1).sum()

0

그러므로 사용 플랫폼 A, 사용 플랫폼 B, 사용 플랫폼 C, 사용 플랫폼 D를 따로 나누지 않고, **사용 플랫폼**이라는 하나의 컬럼으로 합쳤으면 좋겠습니다. 해당 컬럼에는 A, B, C, D 값 중 하나가 들어갑니다.


In [7]:
platform.loc[platform["사용 플랫폼 A"] == 1, "사용 플랫폼"] = "A"
platform.loc[platform["사용 플랫폼 B"] == 1, "사용 플랫폼"] = "B"
platform.loc[platform["사용 플랫폼 C"] == 1, "사용 플랫폼"] = "C"
platform.loc[platform["사용 플랫폼 D"] == 1, "사용 플랫폼"] = "D"

platform[["사용 플랫폼", "사용 플랫폼 A", "사용 플랫폼 B", "사용 플랫폼 C", "사용 플랫폼 D"]].head()

,사용 플랫폼,사용 플랫폼 A,사용 플랫폼 B,사용 플랫폼 C,사용 플랫폼 D
0,D,0,0,0,1
1,D,0,0,0,1
2,D,0,0,0,1
3,D,0,0,0,1
4,D,0,0,0,1


### 2. 작품 분석

다음의 데이터를 활용하여 insights를 얻고 싶습니다.

  * 1 : label. 해당 유저가 목록에 진입하고 1시간 이내에 구매했는지 여부 (이하 구매 여부)
  * 2 : 사용 플랫폼 A
  * 3 : 사용 플랫폼 B
  * 4 : 사용 플랫폼 C
  * 5 : 사용 플랫폼 D
  * 7 : 작품을 나타내는 해쉬 (이하 작품 번호)
  * 112 : 구매할 때 필요한 코인 (이하 구매 코인)
 

여기서 코인 1개의 가격은 100원이라고 가정합니다.

In [8]:
comics = data[[0, 1, 2, 3, 4, 6, 111]].copy()
comics.columns = ["구매 여부", "사용 플랫폼 A", "사용 플랫폼 B", "사용 플랫폼 C", "사용 플랫폼 D", "작품 번호", "구매 코인"]

print(comics.shape)
comics.head()

(650965, 7)


,구매 여부,사용 플랫폼 A,사용 플랫폼 B,사용 플랫폼 C,사용 플랫폼 D,작품 번호,구매 코인
0,0,0,0,0,1,001C9D9B,4
1,0,0,0,0,1,001C9D9B,4
2,0,0,0,0,1,001C9D9B,4
3,0,0,0,0,1,001C9D9B,4
4,0,0,0,0,1,002B4BDE,3


In [9]:
comics.loc[comics["사용 플랫폼 A"] == 1, "사용 플랫폼"] = "A"
comics.loc[comics["사용 플랫폼 B"] == 1, "사용 플랫폼"] = "B"
comics.loc[comics["사용 플랫폼 C"] == 1, "사용 플랫폼"] = "C"
comics.loc[comics["사용 플랫폼 D"] == 1, "사용 플랫폼"] = "D"

comics[["사용 플랫폼", "사용 플랫폼 A", "사용 플랫폼 B", "사용 플랫폼 C", "사용 플랫폼 D"]].head()

,사용 플랫폼,사용 플랫폼 A,사용 플랫폼 B,사용 플랫폼 C,사용 플랫폼 D
0,D,0,0,0,1
1,D,0,0,0,1
2,D,0,0,0,1
3,D,0,0,0,1
4,D,0,0,0,1


** 5. 총 작품의 갯수를 구해주세요. **

In [10]:
comics["작품 번호"].unique().shape

(3559,)

** 6. 가장 많이 구매한 작품 Top 10을 찾아주세요. **

In [14]:
# 구매한 작품 기준으로 계산해야 합니다.
purchased_comics = comics[comics["구매 여부"] == 1]

purchased_comics["작품 번호"].value_counts().head(10)

D619AC7D    5684
00A49090    5298
F0A27F22    3933
85803B60    3894
EC612BAF    3660
621BF66D    3318
BEE0E409    3298
46F57FFA    3103
6983846D    3035
707F7107    2623
Name: 작품 번호, dtype: int64

** 7. 정 반대로, 가장 적게 구매한 작품 Top10을 구해주세요. **

In [15]:
purchased_comics["작품 번호"].value_counts().tail(10)

1D5BD10D    1
A597E505    1
F9D79F8A    1
FFE303DC    1
C0B1FB06    1
D5FC98F7    1
5C48DB79    1
9184B7FA    1
B1B45524    1
F033AB37    1
Name: 작품 번호, dtype: int64

** 8. 구매 코인과 코인 가격을 고려하여, 가장 높은 매출을 달성한 작품 Top 10을 구해주세요. **

한 번 구매했을 때 가격은 구매 코인 * 가격(100 원)이라고 볼 수 있습니다. 가령 구매 코인이 4라면 가격은 400 원이 됩니다.

In [16]:
price = 100

comics["가격"] = comics["구매 코인"] * price

# 구매하지 않은 사람은 가격이 0원이 됩니다.
comics.loc[comics["구매 여부"] == 0, "가격"] = 0

print(comics.shape)
comics[["구매 여부", "구매 코인", "가격"]].head()

(650965, 9)


,구매 여부,구매 코인,가격
0,0,4,0
1,0,4,0
2,0,4,0
3,0,4,0
4,0,3,0


In [17]:
sales = comics.groupby("작품 번호")["가격"].sum()
sales = sales.sort_values(ascending=False)

sales.head(10)

작품 번호
D619AC7D    1705200
00A49090    1589400
F0A27F22    1179900
85803B60    1168200
EC612BAF    1098000
621BF66D     995400
BEE0E409     989400
46F57FFA     930900
6983846D     910500
707F7107     786900
Name: 가격, dtype: int64

** 9. 몇몇 주요 작품들의 사용 플랫폼별 구매 횟수를 구해주세요 **

다음의 다섯 개 아이디(**D91D1B4D**, **72B32A1F**, **06138BC5**, **7F7DDD7D**, **DBB99DB2**)를 주요 작품으로 간주합니다.

해당 다섯 개의 작품이 사용 플랫폼 A~D에서 구매한 횟수를 각각 구해주세요.

In [18]:
tab = pd.crosstab(comics["사용 플랫폼"], comics["작품 번호"])
tab

작품 번호,001C9D9B,002B4BDE,0030E221,00343701,003C31DA,004259EF,004D740D,00545B4F,00548AD2,00563F3B,...,FF7259D4,FF77B8FC,FFA39014,FFB6A968,FFB6E9B2,FFC5BE2E,FFD52F3C,FFE303DC,FFEABD22,FFF1FD1F
사용 플랫폼,,,,,,,,,,,,,,,,,,,,,
A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,13,0,1,0
B,1,9,0,0,0,0,1,0,0,0,...,0,10,2,0,0,0,22,0,12,0
C,96,120,1,1,1,0,0,47,0,1,...,1,247,99,5,0,0,2,1,3,1
D,175,451,4,3,2,1,0,80,2,0,...,0,512,313,7,0,2,9,0,13,0


In [19]:
tab[["D91D1B4D", "72B32A1F", "06138BC5", "7F7DDD7D", "DBB99DB2"]]

작품 번호,D91D1B4D,72B32A1F,06138BC5,7F7DDD7D,DBB99DB2
사용 플랫폼,,,,,
A,0,198,0,2,0
B,20,257,28,31,1
C,423,62,177,0,131
D,1050,272,391,4,249


** 10. 100회 이상 구매한 작품 중, 가장 구매 확률이 높은 작품 top 10을 찾아주세요. **

팁: 세 개를 동시에 시도하지 않고, 1) 100회 이상 구매하지 않은 작품을 제거한 뒤, 2) 1번에서 작품당 구매확률을 구하고, 3) 2번에서 구매 확률로 정렬하는 방식으로 일을 쪼개서 작업하면 조금 더 쉽게 구현할 수 있습니다.

In [20]:
# 100회 이상 구매한 작품만 필터링
top_comics = comics["작품 번호"].value_counts()
top_comics = top_comics[top_comics >= 100]
top_comics = top_comics.index

print(len(top_comics))
top_comics

567


Index(['6983846D', '00A49090', '0365FD34', '621BF66D', 'D619AC7D', '0757A410',
       '37C9A863', '85803B60', 'AC627AB1', 'D8CD8806',
       ...
       'C21EDEEA', 'C399EE1A', 'C45147DE', '9CFDF10E', '0E9821BF', '1AA27FD7',
       'AC1DD209', '33E16A43', '6364D3F0', 'DF9D9F40'],
      dtype='object', length=567)

In [21]:
top_comics = comics[comics["작품 번호"].isin(top_comics)]

print(top_comics.shape)
top_comics.head()

(624747, 9)


,구매 여부,사용 플랫폼 A,사용 플랫폼 B,사용 플랫폼 C,사용 플랫폼 D,작품 번호,구매 코인,사용 플랫폼,가격
0,0,0,0,0,1,001C9D9B,4,D,0
1,0,0,0,0,1,001C9D9B,4,D,0
2,0,0,0,0,1,001C9D9B,4,D,0
3,0,0,0,0,1,001C9D9B,4,D,0
4,0,0,0,0,1,002B4BDE,3,D,0


In [22]:
# 작품당 구매 확률을 계산
tab = pd.crosstab(top_comics["구매 여부"], top_comics["작품 번호"])
tab = tab.apply(lambda r: r / r.sum(), axis=0)
tab

작품 번호,001C9D9B,002B4BDE,00545B4F,00A49090,00EC53C4,01386BD6,01BA9398,01F78BE6,0266E33D,02824AF6,...,F90F2ACA,F980E412,FA1480CE,FAA710EA,FBD7939D,FC15FECC,FC490CA4,FC6FC140,FF77B8FC,FFA39014
구매 여부,,,,,,,,,,,,,,,,,,,,,
0,0.672794,0.617241,0.748031,0.396721,0.463235,0.586691,0.675585,0.726562,0.527972,0.974952,...,0.785124,0.841945,0.578001,0.498457,0.350195,0.887435,0.467033,1.0,0.656697,0.364734
1,0.327206,0.382759,0.251969,0.603279,0.536765,0.413309,0.324415,0.273438,0.472028,0.025048,...,0.214876,0.158055,0.421999,0.501543,0.649805,0.112565,0.532967,0.0,0.343303,0.635266


In [23]:
# 구매 확률이 높은 쪽으로 정렬
top = tab.loc[1].sort_values(ascending=False)
top10 = top.head(10)
top10

작품 번호
149E9677    0.873250
F0935E4C    0.843168
F0A27F22    0.840924
06138BC5    0.825503
BCA82E41    0.806028
3CB79548    0.804795
AD13A2A0    0.801153
10261288    0.799733
D18F655C    0.793467
69ADC1E1    0.786813
Name: 1, dtype: float64

### 3. 주요 작품 분석

다음의 데이터를 활용하여 insights를 얻고 싶습니다.

  * 1 : label. 해당 유저가 목록에 진입하고 1시간 이내에 구매했는지 여부 (이하 구매 여부)
  * 7 : 작품을 나타내는 해쉬 (이하 작품 번호)
  * 11-110 : 주요 작품 구매 여부. (이하 주요 작품 1 ~ 주요 작품 100)

In [24]:
import numpy as np

related_comices = data.loc[:, 10:109].astype(np.int32)
related_comices.columns = [f"주요 작품 {i+1}" for i in range(related_comices.shape[1])]

print(related_comices.shape)
related_comices.head()

(650965, 100)


,주요 작품 1,주요 작품 2,주요 작품 3,주요 작품 4,주요 작품 5,주요 작품 6,주요 작품 7,주요 작품 8,주요 작품 9,주요 작품 10,...,주요 작품 91,주요 작품 92,주요 작품 93,주요 작품 94,주요 작품 95,주요 작품 96,주요 작품 97,주요 작품 98,주요 작품 99,주요 작품 100
0,0,11,0,0,1,12,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,25,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,34,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,6,0,0,0,0,0,...,0,0,0,0,0,6,0,0,0,0


** 11. 가장 많은 횟수를 구매한 주요 작품 Top 10을 찾아주세요. **

In [25]:
related_comices.sum(axis=0).sort_values(ascending=False).head(10)

주요 작품 10    2287592
주요 작품 2     2207155
주요 작품 1     1707746
주요 작품 5     1647832
주요 작품 6     1526819
주요 작품 45    1452128
주요 작품 8     1438426
주요 작품 37    1336681
주요 작품 16    1262415
주요 작품 11    1256666
dtype: int64

** 12. 구매 여부를 기준으로, 구매하지 않은 사용자는 배제하고 구매한 사용자 기준으로 가장 많은 횟수를 구매한 주요 작품 Top 10을 찾아주세요. **

In [26]:
related_comices["구매 여부"] = data[0]

purchased_related_comices = related_comices[related_comices["구매 여부"] == 1]

print(purchased_related_comices.shape)
purchased_related_comices.head()

(221793, 101)


,주요 작품 1,주요 작품 2,주요 작품 3,주요 작품 4,주요 작품 5,주요 작품 6,주요 작품 7,주요 작품 8,주요 작품 9,주요 작품 10,...,주요 작품 92,주요 작품 93,주요 작품 94,주요 작품 95,주요 작품 96,주요 작품 97,주요 작품 98,주요 작품 99,주요 작품 100,구매 여부
216289,0,34,0,0,12,0,0,0,0,38,...,10,0,0,0,0,0,0,0,0,1
216290,0,10,0,0,10,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
216291,0,4,0,0,34,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
216292,0,4,0,0,34,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
216293,0,4,0,0,34,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [27]:
purchased_related_comices.sum(axis=0).sort_values(ascending=False).head(10)

주요 작품 10    1326860
주요 작품 2     1211597
주요 작품 5      952626
주요 작품 8      950861
주요 작품 1      931839
주요 작품 45     870640
주요 작품 6      859250
주요 작품 37     845605
주요 작품 3      791907
주요 작품 4      765360
dtype: int64

** 13. 구매 횟수와 상관 없이, 구매 확률이 가장 높은 주요 작품 Top 10을 찾아주세요. **

가령 구매를 20번 하건 30번 하건 전부 1번으로 가정 합니다. 또 다른 예를 들자면, 전체 650,965 개의 데이터에서 주요 작품 1번을 6,509번 구매했다면 구매 확률은 0.01(1%)로 가정합니다.

In [28]:
related_comices_binary = related_comices.astype('bool').astype('int')
related_comices_binary.mean(axis=0).sort_values(ascending=False).head(10)

구매 여부       0.340714
주요 작품 2     0.208850
주요 작품 10    0.164792
주요 작품 18    0.159105
주요 작품 13    0.158565
주요 작품 5     0.156363
주요 작품 11    0.144926
주요 작품 1     0.133337
주요 작품 41    0.128431
주요 작품 6     0.127589
dtype: float64

** 14. 작품 번호별로, 구매 횟수가 가장 높은 주요 작품(주요 작품 1 ~ 100)의 Top 10을 찾아주세요. **

또한 작품 번호는 누적 구매 수가 가장 많은 순으로 정렬, 주요 작품도 누적 구매 수가 가장 많은 순으로 정렬합니다.

In [29]:
related_comices["작품 번호"] = data[6]

table = pd.pivot_table(related_comices,
                       index="작품 번호",
                       values=[f"주요 작품 {i+1}" for i in range(100)],
                       aggfunc=np.sum)

table.head(10)

,주요 작품 1,주요 작품 10,주요 작품 100,주요 작품 11,주요 작품 12,주요 작품 13,주요 작품 14,주요 작품 15,주요 작품 16,주요 작품 17,...,주요 작품 90,주요 작품 91,주요 작품 92,주요 작품 93,주요 작품 94,주요 작품 95,주요 작품 96,주요 작품 97,주요 작품 98,주요 작품 99
작품 번호,,,,,,,,,,,,,,,,,,,,,
001C9D9B,974,2506,712,706,101,1389,57,196,1447,660,...,83,19,633,224,680,50,87,1,336,150
002B4BDE,3184,4895,1271,1941,281,2795,221,1727,2932,1487,...,120,67,1295,185,1966,172,211,111,539,403
0030E221,21,1,0,0,0,0,0,23,0,0,...,0,0,0,62,17,0,0,0,2,0
00343701,82,31,56,76,0,30,0,1,22,0,...,14,0,15,7,19,0,0,0,0,1
003C31DA,39,1,0,35,0,0,0,0,0,0,...,0,0,0,0,7,0,0,0,0,0
004259EF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,90,0,0,0,0,0,2
004D740D,0,0,0,0,0,13,0,0,0,0,...,0,0,0,0,14,0,0,0,0,0
00545B4F,354,919,212,340,52,607,109,196,704,248,...,37,0,351,20,187,16,59,104,6,71
00548AD2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,3,0,0,0,0,0


In [30]:
index = table.sum(axis=1).sort_values(ascending=False).index
columns = table.sum(axis=0).sort_values(ascending=False).index

table.loc[index, columns].head(10)

,주요 작품 10,주요 작품 2,주요 작품 1,주요 작품 5,주요 작품 6,주요 작품 45,주요 작품 8,주요 작품 37,주요 작품 16,주요 작품 11,...,주요 작품 100,주요 작품 42,주요 작품 92,주요 작품 78,주요 작품 66,주요 작품 81,주요 작품 43,주요 작품 85,주요 작품 86,주요 작품 95
작품 번호,,,,,,,,,,,,,,,,,,,,,
EC612BAF,57858,13844,15399,9841,8580,42383,27438,65582,10419,20881,...,1640,1411,9154,8266,10469,11648,1048,1607,10050,14863
F0A27F22,52073,17014,18739,7965,8677,41381,20972,71215,8319,24475,...,1267,1202,8566,11880,16283,19814,4572,3954,12850,18805
D619AC7D,85663,59265,25414,29276,43413,58467,4703,27225,39953,28171,...,5166,9233,11799,3779,4301,5532,719,1156,3113,6096
BEE0E409,34301,17529,17577,7573,10225,24157,15093,31700,11484,17157,...,1523,2040,4775,6842,7461,7010,1977,1695,6214,8840
3942C7FE,54135,30495,12685,18184,26095,39228,2503,19942,19473,12785,...,3673,4946,11992,2318,3720,3313,412,692,2039,3453
4A99FE0F,2075,1812,5925,2493,1058,244,61023,11003,254,5756,...,284,70,88,3936,2741,1920,2839,2763,4007,1405
85803B60,43397,118009,30305,34708,37410,18210,2024,9069,23566,18695,...,5722,6869,4303,1787,1799,1966,687,1374,1245,1519
707F7107,39673,42275,17767,29959,36361,27775,1680,9533,28235,16584,...,7224,17028,5862,2015,1572,1693,694,1149,1282,1149
00A49090,32664,45213,19903,156981,32059,14348,1160,5628,15787,12186,...,11005,6027,3474,1107,1008,953,891,775,667,463
